# Reducing ICESat-2 data files

* Select files of interest (segment and time)
* Select area of interest (subset lon/lat)
* Reduce selected files with variables of interest
* Filter data and separate tracks into asc/des
* Process/Read each file in parallel
* Plot some data to check everything went well

## How ICESat-2 files are organized 

ICESat-2 ground tracks are subsetted into granules (individual files)

Granules are then grouped into latitudinal bands (segments)

![Segments](segments.png "Latitudinal bands (Segments)")


File naming convention:

`ATL06_20181120202321_08130101_001_01.h5`

`ATL06_[yyyymmdd][hhmmss]_[RGTccss]_[vvv_rr].h5`

where

`ATL_06` => L3A Land Ice product    

`yyyymmdd` => Year, month, day of data acquisition    

`hhmmss` => Hour, minute, second of data acquisition   

`RGT` => Reference Ground Track    

`cc` => Cycle Number   

`ss` => Segment number (latitude band)   

`vvv_rr` => Version and revision numbers  

## Select files of interest

We will use the **file name** info for this (no need to open the files).  

Alternatively, we could retrieve this info from the **Metadata**.

To select files withint a time interval and segment, all we need is:

`yyyymmdd, hhmmss, ss`

Let's firt get a list with all file names

In [1]:
#from utils import *

def list_files_local(path):
    """ Get file list form local folder. """
    from glob import glob
    return glob(path)


def list_files_ssh(path, host, user, pwd):
    """ Get file list from remote folder usgin SSH. """
    import paramiko  # pip install paramiko

    # Create an SSH client instance.
    client = paramiko.SSHClient()

    # Create a 'host_keys' object
    # and load the local known hosts  
    host_keys = client.load_system_host_keys()

    # Connect to our client w/remote machine credentials
    client.connect(host, username=user, password=pwd)

    # Execute command on remote system,
    # and get input, output and error variables
    stdin, stdout, stderr = client.exec_command('ls '+path)

    # Iterate over stdout
    files = [line.strip('\n') for line in stdout]

    # Close the connection to client
    client.close()
    return files
    
    
def list_files_s3(path):
    """ Get file list from Amazon S3. """
    # code for Amazon S3 here
    pass


if 1:
    files = list_files_local('data/*.h5')
    
elif 0:    
    path = '/u/devon-r2/shared_data/icesat2/atl06/rel205/raw/*.h5'
    host, user, pwd = 'devon.jpl.nasa.gov', 'paolofer', '********'

    files = list_files_ssh(path, host, user, pwd)
    
else:
    files = list_files_s3(path)
    
print('Total number of files:', len(files))

Total number of files: 0


Filter file names by segment and time interval

In [27]:
import os
import datetime as dt

# File name format: ATL06_[yyyymmdd][hhmmss]_[RGTccss]_[vvv_rr].h5

#NOTE: Need to simplify this function
def time_from_fname(fname):
    t = fname.split('_')[1]
    y, m , d, h, mn, s = t[:4], t[4:6], t[6:8], t[8:10], t[10:12], t[12:14]
    time = dt.datetime(int(y), int(m), int(d), int(h), int(mn), int(s))
    return time


def segment_from_fname(fname):
    s = fname.split('_')[2]
    return int(s[-2:])


def select_files(files, segments=[10,11,12], t1=(2019,1,1), t2=(2019,2,1)):
    t1 = dt.datetime(*t1)
    t2 = dt.datetime(*t2)
    files_out = []
    for f in files:
        fname = os.path.basename(f)
        time = time_from_fname(fname)
        segment = segment_from_fname(fname)
        if t1 <= time <= t2 and segment in segments:
            files_out.append(f)
    return files_out

In [28]:
files = select_files(files, segments=[10,11,12], t1=(2019,1,1), t2=(2019,2,1))

for f in files[:5]: print(f)
print('Number of files:', len(files))

/u/devon-r2/shared_data/icesat2/atl06/rel205/raw/ATL06_20190101001723_00540210_205_01.h5
/u/devon-r2/shared_data/icesat2/atl06/rel205/raw/ATL06_20190101002504_00540211_205_01.h5
/u/devon-r2/shared_data/icesat2/atl06/rel205/raw/ATL06_20190101003047_00540212_205_01.h5
/u/devon-r2/shared_data/icesat2/atl06/rel205/raw/ATL06_20190101015140_00550210_205_01.h5
/u/devon-r2/shared_data/icesat2/atl06/rel205/raw/ATL06_20190101015921_00550211_205_01.h5
Number of files: 1388


Let's download the selected files

In [4]:
# code for downloading selected files to -> data/
# The files should already be dowloaded. This cell should not be executed.

## Reduce ICESat-2 files

***
**NOTE:** 

**This is neither the only nor the best way to handled ICESat-2 data files.**

**This is *one* way that works well for large-scale processing (e.g. full continent) on parallel machines (e.g. HPC clusters).**

**The idea is to (a) simplify the I/O of a complex workflow and (b) take advantage of embrrasingly parallelization.**
***

Let's check the ICESat-2 file structure

In [29]:
!h5ls -r data/*.h5

data/*.h5: unable to open file


Let's code a simple reader that:

- Select variables of interest (x, y, t, h...)  
- Filter data points based on quality flag and bbox   
- Separate into beams and ascending/descending tracks  
- Save data to a simpler HDF5 structure

In [26]:
#from utils import gps2dyr  <=== NOT WORKING!

def read_atl06(ifile, bbox=None):
    """ 
    Read 1 ATL06 file and output 12 reduced files. 
    
    Extract variables of interest and separate the ATL06 file 
    into each beam (ground track) and ascending/descending orbits.
    """

    # Each beam is a group
    group = ['./gt1l','./gt1r','./gt2l','./gt2r','./gt3l','./gt3r']

    # Loop trough beams
    for k,g in enumerate(group):
    
        #-----------------------------------#
        # 1) Read in data for a single beam #
        #-----------------------------------#
    
        # Load variables into memory (more can be added!)
        with h5py.File(ifile, 'r') as fi:
            lat = fi[g+'/land_ice_segments/latitude'][:]
            lon = fi[g+'/land_ice_segments/longitude'][:]
            h_li = fi[g+'/land_ice_segments/h_li'][:]
            s_li = fi[g+'/land_ice_segments/h_li_sigma'][:]
            t_dt = fi[g+'/land_ice_segments/delta_time'][:]
            flag = fi[g+'/land_ice_segments/atl06_quality_summary'][:]
            s_fg = fi[g+'/land_ice_segments/fit_statistics/signal_selection_source'][:]
            snr = fi[g+'/land_ice_segments/fit_statistics/snr_significance'][:]
            h_rb = fi[g+'/land_ice_segments/fit_statistics/h_robust_sprd'][:]
            dac = fi[g+'/land_ice_segments/geophysical/dac'][:]
            f_sn = fi[g+'/land_ice_segments/geophysical/bsnow_conf'][:]
            dh_fit_dx = fi[g+'/land_ice_segments/fit_statistics/dh_fit_dx'][:]
            tide_earth = fi[g+'/land_ice_segments/geophysical/tide_earth'][:]
            tide_load = fi[g+'/land_ice_segments/geophysical/tide_load'][:]
            tide_ocean = fi[g+'/land_ice_segments/geophysical/tide_ocean'][:]
            tide_pole = fi[g+'/land_ice_segments/geophysical/tide_pole'][:]
            t_ref = fi['/ancillary_data/atlas_sdp_gps_epoch'][:]
            rgt = fi['/orbit_info/rgt'][:] * np.ones(len(lat))

        #---------------------------------------------#
        # 2) Filter data according region and quality #
        #---------------------------------------------#
        
        # Select a region of interest
        if bbox:
            lonmin, lonmax, latmin, latmax = bbox
            bbox_mask = (lon >= lonmin) & (lon <= lonmax) & (lat >= latmin) & (lat <= latmax)
        else:
            bbox_mask = np.ones_like(lat, dtype=bool)  # get all
            
        # Only keep good data, and data inside bbox
        mask = (flag == 0) & (np.abs(h_li) < 10e3) & (bbox_mask > 0)
        
        # Update variables
        lat, lon, h_li, s_li, t_dt, h_rb, s_fg, snr, q_flag, f_sn, \
            tide_earth, tide_load, tide_ocean, tide_pole, dac, rgt = \
                lat[mask], lon[mask], h_li[mask],s_li[mask], t_dt[mask], h_rb[mask], \
                s_fg[mask], snr[mask], q_flag[mask], f_sn[mask], tide_earth[mask], \
                tide_load[mask], tide_ocean[mask], tide_pole[mask], dac[mask], rgt[mask]

        # Test for no data
        if len(h_li) == 0: return

        #-------------------------------------#
        # 3) Convert time and separate tracks #
        #-------------------------------------#
        
        # Time in GPS seconds (secs sinde 1980...)
        t_gps = t_ref + t_dt

        # Time in decimal years
        t_year = gps2dyr(t_gps)

        # Determine orbit type
        i_asc, i_des = track_type(t_year, lat)
        
        #-----------------------#
        # 4) Save selected data #
        #-----------------------#
        
        # Define output file name
        ofile = ifile.replace('.h5', '_'+g[2:]+'.h5')
        
        #NOTE: Asc/Des can be saved in a single file w/index=0|1
                
        # Save ascending track data
        if len(lat[i_asc]) > 1:
            fasc = ofile.replace('.h5', '_A.h5')
            with h5py.File(fasc, 'w') as fa:
                fa['orbit'] = orb[i_asc][:]
                fa['lon'] = lon[i_asc][:]
                fa['lat'] = lat[i_asc][:]
                fa['h_elv'] = h_li[i_asc][:]
                fa['s_elv'] = s_li[i_asc][:]
                fa['t_year'] = t_li[i_asc][:]
                fa['h_rb'] = h_rb[i_asc][:]
                fa['s_fg'] = s_fg[i_asc][:]
                fa['snr'] = snr[i_asc][:]
                fa['q_flg'] = q_flag[i_asc][:]
                fa['f_sn'] = f_sn[i_asc][:]
                fa['t_sec'] = t_gps[i_asc][:]
                fa['tide_load'] = tide_load[i_asc][:]
                fa['tide_ocean'] = tide_ocean[i_asc][:]
                fa['tide_pole'] = tide_pole[i_asc][:]
                fa['tide_earth'] = tide_earth[i_asc][:]
                fa['dac'] = dac[i_asc][:]
                fa['rgt'] = rgt[i_asc][:]                

                print('asc ->', fasc)

        # Save desending track data
        if len(lat[i_des]) > 1:
            fdes = ofile.replace('.h5', '_D.h5')
            with h5py.File(fdes, 'w') as fd:
                fd['orbit'] = orb[i_des][:]
                fd['lon'] = lon[i_des][:]
                fd['lat'] = lat[i_des][:]
                fd['h_elv'] = h_li[i_des][:]
                fd['s_li'] = s_li[i_des][:]
                fd['t_year'] = t_li[i_des][:]
                fd['h_rb'] = h_rb[i_des][:]
                fd['s_fg'] = s_fg[i_des][:]
                fd['snr'] = snr[i_des][:]
                fd['q_flg'] = q_flag[i_des][:]
                fd['f_sn'] = f_sn[i_des][:]
                fd['t_sec'] = t_gps[i_des][:]
                fd['tide_load'] = tide_load[i_des][:]
                fd['tide_ocean'] = tide_ocean[i_des][:]
                fd['tide_pole'] = tide_pole[i_des][:]
                fd['tide_earth'] = tide_earth[i_des][:]
                fd['dac'] = dac[i_des][:]
                fd['rgt'] = rgt[i_des][:]
                
                print('des ->', fdes)
                

* Merge beams for crossover analysis?

## Simple parallelization

In [4]:
njobs = 16

files = [] #list_files_local(path)

#NOTE: Using Kamb bounding box for now
bbox = [-1124782, 81623, -919821, -96334]

if njobs == 1:
    print('running in serial ...')
    [read_atl06(f, bbox) for f in files]

else:
    print('running in parallel (%d jobs) ...' % njobs)
    from joblib import Parallel, delayed
    Parallel(n_jobs=njobs, verbose=5)(delayed(read_atl06)(f, bbox) for f in files)


running in parallel (16 jobs) ...


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   0 out of   0 | elapsed:    0.0s finished


Let's check our created files

In [28]:
!ls data/*.h5

ls: data/*.h5: No such file or directory


In [27]:
!h5ls -r data/*gt2r_A.h5

data/*gt2r_A.h5: unable to open file


## Plot some data to check

* Plot points within region to check (map and scatter)
* Plot tracks/beams to check (map and profiles)
* Show single program with all of the above in one go (`readatl06.py`)